In [1]:
import numpy as np
import pandas as pd
import pymysql  
from sqlalchemy import create_engine

In [2]:
#Databse connnection
cnx = create_engine('mysql+pymysql://root:@localhost/athena')

In [3]:
kiva_loans=pd.read_sql("SELECT * FROM kiva_loans", cnx)
kiva_loans.head(2)

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300,300,0,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12,12,,female,irregular,1/1/2014
1,653053,575,575,0,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11,14,,"female, female",irregular,1/1/2014


In [5]:
kiva_mpi_region_locations=pd.read_sql("SELECT * FROM kiva_mpi_region_locations", cnx)
kiva_mpi_region_locations.head(3)

,LocationName,ISO,country,region,world_region,MPI,geo,lat,lon
0,"Badakhshan, Afghanistan",AFG,Afghanistan,Badakhshan,South Asia,0.387,"(36.7347725, 70.81199529999999)",36.734772,70.811995
1,"Badghis, Afghanistan",AFG,Afghanistan,Badghis,South Asia,0.466,"(35.1671339, 63.7695384)",35.167134,63.769538
2,"Baghlan, Afghanistan",AFG,Afghanistan,Baghlan,South Asia,0.300,"(35.8042947, 69.2877535)",35.804295,69.287753


In [6]:
loan_theme_ids=pd.read_sql("SELECT * FROM loan_theme_ids", cnx)

,id,Loan Theme ID,Loan Theme Type,Partner ID
0,638631,a1050000000skGl,General,151
1,640322,a1050000000skGl,General,151
2,641006,a1050000002X1ij,Higher Education,160


In [7]:
#Save As CSV
loan_themes_by_region=pd.read_sql("SELECT * FROM loan_themes_by_region", cnx)

###### Kiva_loans combined dataset

In [8]:
kiva_loans_combined_dataset=pd.merge(kiva_loans, kiva_mpi_region_locations[["region","MPI"]], on="region", how="left")
kiva_loans_combined_dataset1 = pd.merge(kiva_loans_combined_dataset, loan_theme_ids[["id", "Partner ID"]], on="id", how="left")
kiva_loans_combined_dataset_whole = pd.merge(kiva_loans_combined_dataset1, loan_themes_by_region[["Partner ID", "Field Partner Name"]], on="Partner ID", how="left")

In [ ]:
kiva_loans_combined_dataset_whole.to_csv("kiva_loans_dataset.csv", index=False)

### Sectoral impact analytics

In [12]:
total_funded_per_sector = kiva_loans.groupby('sector')['loan_amount'].sum().sort_values(ascending=False)

# loans per sector
num_loans_per_sector = kiva_loans['sector'].value_counts().sort_values(ascending=False)

# Average loan amount per sector
average_loan_amount_per_sector = kiva_loans.groupby('sector')['loan_amount'].mean().sort_values(ascending=False)

print("Amountfunded per sector:")
print(total_funded_per_sector)
print("\nNumber of loans per sector:")
print(num_loans_per_sector)
print("\nAverage loan amount per sector:")
print(average_loan_amount_per_sector)

Amountfunded per sector:
sector
Food              2441650
Agriculture       2315925
Retail            2043125
Services           982350
Clothing           720225
Housing            601875
Education          485150
Transportation     359450
Arts               225575
Personal Use       199625
Construction       148700
Manufacturing      141975
Health              94875
Entertainment       21500
Wholesale           10925
Name: loan_amount, dtype: int64

Number of loans per sector:
Food              2652
Agriculture       2591
Retail            2337
Services           909
Housing            678
Clothing           667
Education          663
Transportation     429
Personal Use       287
Arts               191
Manufacturing      167
Construction       157
Health              83
Entertainment       15
Wholesale            8
Name: sector, dtype: int64

Average loan amount per sector:
sector
Entertainment     1433.333333
Wholesale         1365.625000
Arts              1181.020942
Health         